In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install --pre higlass-python

## Synced heatmaps

In [ ]:
import higlass as hg

# Configure remote data sources (tilesets)
tileset1 = hg.remote(
    uid="CQMd6V_cRw6iCI_-Unl3PQ",
    server="https://higlass.io/api/v1/",
    name="Rao et al. (2014) GM12878 MboI (allreps) 1kb",
)

tileset2 = hg.remote(
    uid="QvdMEvccQuOxKTEjrVL3wA",
    server="https://higlass.io/api/v1/",
    name="Rao et al. (2014) K562 MboI (allreps) 1kb",
)

# Create a HeatmapTrack for each tileset
track1 = tileset1.track("heatmap")
track2 = tileset2.track("heatmap")

# Create two independent Views, one for each heatmap
view1 = hg.view(track1, width=6)
view2 = hg.view(track2, width=6)

# Lock zoom & location for each View
view_lock = hg.lock(view1, view2)

# Concatenate Views side-by-side, and apply synchronization lock
(view1 | view2).locks(view_lock)

In [ ]:
# Lock zoom only for each view
(view1 | view2).locks(zoom=view_lock)

In [ ]:
# Lock location only for each view
(view1 | view2).locks(location=view_lock)

In [ ]:
# Create additional views and synchronize (apply black to white color range)

bw_color_range = ["rgba(255,255,255,1)", "rgba(0,0,0,1)"]
view3 = hg.view(track1.opts(colorRange=bw_color_range), width=6)
view4 = hg.view(track2.opts(colorRange=bw_color_range), width=6)

# Create stacked view configuration and lock views by column
((view1 / view2) | (view3 / view4)).locks(
    hg.lock(view1, view2),
    hg.lock(view3, view4),
)

## Value scale syncing

In [ ]:
# Create Tracks

# creates a hg.Track without a tileset
t1 = hg.track("top-axis")

# Creates a hg.RemoteTileset object
remote_tileset = hg.remote(
    uid="CQMd6V_cRw6iCI_-Unl3PQ",
    server="https://higlass.io/api/v1/",  # optional, "http://higlass.io/api/v1/"
    name="Rao et al. (2014) GM12878 MboI (allreps) 1kb",
)

# Tileset.track() creates a hg.Track object binding the parent Tileset
t2 = remote_tileset.track("heatmap").opts(valueScaleMax=0.5)

# Create Views

# Positional arguments for `hg.view` are overloaded. Keyword arguments are
# forwarded to the creation of the `hg.View`. Layout fields (`x`, `y`,
# `width`, `height`) may also be assigned.

# (1) Track objects (positioning guessed based on track type)
view1 = hg.view(t1, t2, width=6)

# (2) (Track, position) tuple
view2 = hg.view((t1, "top"), t2, width=6)

# (3) hg.Tracks object
view3 = hg.view(hg.Tracks(top=[t1], center=[t2]), width=6)

# (View, Track) tuples -> ScaleValueLock
scale_value_lock = hg.lock((view1, t2), (view2, t2))

(view1 | view2).locks(scale_value_lock)

## Remote heatmaps

In [ ]:
# Initialize data sources
tset1 = hg.remote(
    uid="CQMd6V_cRw6iCI_-Unl3PQ",
    name="Rao et al. (2014) GM12878 MboI (allreps) 1kb",
)

tset2 = hg.remote(
    uid="QvdMEvccQuOxKTEjrVL3wA",
    name="Rao et al. (2014) K562 MboI (allreps) 1kb",
)

# Create a track for each data source
t1 = tset1.track("heatmap", height=300)
t2 = tset2.track("heatmap", height=300)

# Create a custom DividedTrack and modify color scale
t3 = hg.divide(t1, t2).opts(
    colorRange=["blue", "white"],
    valueScaleMin=0.1,
    valueScaleMax=10,
)

# Set initial view domains
domain = (7e7, 8e7)
v1 = hg.view(t1, width=4).domain(x=domain)
v2 = hg.view(t2, width=4).domain(x=domain)
v3 = hg.view(t3, width=4).domain(x=domain)

(v1 | v3 | v2).locks(hg.lock(v1, v2, v3))

## Extract track from another view config

In [ ]:
# Load view config from URL
url = "https://gist.githubusercontent.com/manzt/c2c498dac3ca9804a2b8e4bc1af3b55b/raw/ee8426c9728e875b6f4d65030c61181c6ba29b53/example.json"
config = hg.Viewconf.from_url(url)

# Display viewconfig
config

In [ ]:
# Inspect the Viewconf tracks
for position, track in config.views[0].tracks:
    print(position)  # position in view layout
    print(track)  # python object from `higlass-schema`

In [ ]:
# Extract a specific track from the above Viewconf and modify its properties
gene_annotation_track = config.views[0].tracks.top[0].properties(height=200)

# Display track in isolation
hg.view(gene_annotation_track)

## Local bigWig files

In [ ]:
!wget http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878InputStdSig.bigWig

In [ ]:
tileset = hg.bigwig("wgEncodeSydhTfbsGm12878InputStdSig.bigWig")
hg.view(tileset.track("horizontal-bar"))

## Local cooler files

This section describes how to load cooler files that are on the same filesystem as the Jupyter notebook.

In [ ]:
tileset = hg.cooler("../test/data/Dixon2012-J1-NcoI-R1-filtered.100kb.multires.cool")
hg.view(tileset.track("heatmap"))